In [2]:
import pandas as pd
import numpy as np
import os
import glob
import joblib
from datetime import datetime, timedelta
from pathlib import Path

In [1]:
path = "/ml/algorithmlabs.csv"

In [3]:
file_list = glob.glob(f'{path}/*.csv')
name = pd.read_excel('ml/feature_korean_name.xlsx', sheet_name='attrition')

### DAAV_BASEINFO_DETAIL

In [4]:
print(file_list[4])
bt = pd.read_csv(file_list[4])

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAAV_BASEINFO_DETAIL.csv


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_22500\3824298898.py:2: DtypeWarning: Columns (36,38,39,53,167,171,173) have mixed types. Specify dtype option on import or set low_memory=False.
  bt = pd.read_csv(file_list[4])


In [5]:
# label 세팅 영역으로 21년 1~12월의 재직자 데이터를 참고했을 때, 6개월 후(22년 1~6월)에 누가 퇴사할 위험성이 높은지 예측
bt_data_train = bt[(bt['DT_JOIN']<=20221231) & ((bt['DT_RETIRE'] >= 20220101) | (bt['DS_RETIRE'] == '재직'))]
bt_data_test = bt[(bt['DT_JOIN']<=20221231) & ((bt['DT_RETIRE'] >= 20230101) | (bt['DS_RETIRE'] == '재직'))]
bt_data_train['TEMP'] = 'train'
bt_data_test['TEMP'] = 'test'


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20296\1196069732.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bt_data_train['TEMP'] = 'train'
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20296\1196069732.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bt_data_test['TEMP'] = 'test'


In [6]:

bt_data = pd.concat([bt_data_train, bt_data_test], axis=0)
bt_data = bt_data.reset_index(drop=True)
bt_data['DT_YEAR'] = 2022
bt_data['ID_SABUN'] = bt_data['ID_SABUN'].astype('str')


In [8]:
bt_data = bt_data[["TEMP", "ID_SABUN", "DT_YEAR", "DT_GROUPJOIN", "DT_JOIN", "CD_JOBTYPE", "CD_POSITION", "CD_DEPT", "CD_BONBU", "TY_DEPT", "DT_RETIRE", "DS_RETIRE", "CD_OCCUPATION", "CD_DUTY", 
                   "DT_BIRTH", "DS_BIRTHPLACE", "CD_ADOPTYPE", "YN_HANDICAP", "YN_SUPPORTED", "YN_FOREIGNER", "YN_GENDER", "CD_JOBFAMILY", "CD_JOBFAMILY_JOIN", 
                   "DS_JOBTYPE", "DS_POSITION", "DS_DEPT", "DS_BONBU", "DS_TY_DEPT", "DS_OCCUPATION", "DS_DUTY", "DS_ADOPTYPE", "DS_JOBFAMILY", "DS_JOBFAMILY_JOIN",
                   'DS_HNAME', 'DS_HANDPHONE', 'DS_EMAIL']]

### DACT_USEDVACATION

In [5]:
print(file_list[5])
vat = pd.read_csv(file_list[5])
vat_data = vat.copy()

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DACT_USEDVACATION.csv


In [10]:
vat_data = vat_data[vat_data['YN_COMMONVACATION']!='Y']
vat_data.sort_values(by=['ID_SABUN', 'DT_VACAFROM'], ascending=True, inplace=True)
vat_data = vat_data.reset_index(drop=True)

In [11]:
vat_list = []

for id in vat_data['ID_SABUN'].unique().tolist() : 
    cnt = 1
    temp_vat = vat_data[vat_data['ID_SABUN']==id]
    temp_vat = temp_vat.reset_index(drop=True)
    for j in range(1, len(temp_vat)) : 
        first = datetime(int(str(temp_vat['DT_VACAFROM'][j-1])[0:4]), int(str(temp_vat['DT_VACAFROM'][j-1])[4:6]), int(str(temp_vat['DT_VACAFROM'][j-1])[6:8]))
        second = datetime(int(str(temp_vat['DT_VACAFROM'][j])[0:4]), int(str(temp_vat['DT_VACAFROM'][j])[4:6]), int(str(temp_vat['DT_VACAFROM'][j])[6:8]))
        if (first + timedelta(days=1))==second : 
            cnt += 1
        else : 
            vat_list.append([id, temp_vat['DT_VACAFROM'][j-1], cnt])
            cnt = 1

In [12]:
vacation = pd.DataFrame(vat_list, columns=['ID_SABUN', 'DT_VACAFROM', 'VAT'])
vacation = vacation[vacation['VAT']==1].reset_index(drop=True)

In [13]:
year = []
month = []
for i in range(len(vacation)) : 
    year.append(str(vacation['DT_VACAFROM'][i])[0:4])
    month.append(str(vacation['DT_VACAFROM'][i])[4:6])
    
vacation['DT_YEAR'] = year
vacation['DT_MONTH'] = month

In [14]:
date_list = []

for id in vacation['ID_SABUN'].unique().tolist() : 
    temp_data = vacation[vacation['ID_SABUN']==id]
    value_list = temp_data[temp_data['DT_YEAR']=='2022']['DT_MONTH'].value_counts()
    for month, cnt in zip (value_list.index, value_list.values) :
        date_list.append([id, 2022, month, cnt])
        
used_data = pd.DataFrame(date_list, columns=['ID_SABUN', 'DT_YEAR', 'DT_MONTH', 'VAT_CNT'])
used_data['ID_SABUN'] = used_data['ID_SABUN'].astype('str')

In [15]:
train_month = ['04', '05', '06']
inf_month = ['10', '11', '12']

vat_month = []

for i in range(len(bt_data)) : 
    vat_month_list = [] 
    if bt_data['TEMP'][i] == 'train' : 
        temp_data = used_data[used_data['ID_SABUN']==bt_data['ID_SABUN'][i]]
        for month in train_month : 
            if month in temp_data['DT_MONTH'].unique().tolist() : 
                vat_month_list.append(temp_data[temp_data['DT_MONTH']==month]['VAT_CNT'].values[0])
            else : 
                vat_month_list.append(0)
    else : 
        temp_data = used_data[used_data['ID_SABUN']==bt_data['ID_SABUN'][i]]
        for month in inf_month : 
            if month in temp_data['DT_MONTH'].unique().tolist() : 
                vat_month_list.append(temp_data[temp_data['DT_MONTH']==month]['VAT_CNT'].values[0])
            else : 
                vat_month_list.append(0)
    vat_month.append(vat_month_list)

bt_data[['VAT_THREE_MONTH_AGO', 'VAT_TWO_MONTH_AGO', 'VAT_ONE_MONTH_AGO']] = vat_month

### DAJT_FINAL_RESULT, DAKT_MBO_USER

In [6]:
print(file_list[8])
wpe = pd.read_csv(file_list[8])
wpe_data = wpe.copy()
wpe_data = wpe_data[(wpe_data['CD_CORP']=='A101') & (wpe_data['DT_YEAR']>=2018)]

print(file_list[9])
mbo = pd.read_csv(file_list[9])
mbo_data = mbo.copy()
mbo_data = mbo_data[(mbo_data['CD_CORP']=='A101') & (mbo_data['DT_YEAR']>=2018)]

wpe_data = wpe_data[['ID_SABUN', 'DT_YEAR', 'NO_POINT_TOTAL_FIN', 'DS_POINT_TOTAL_FIN']]
mbo_data = mbo_data[['CD_USER', 'DT_YEAR', 'NO_FINAL_POINT', 'DS_FINAL_POINT']]
mbo_data.rename(columns = {'CD_USER' : 'ID_SABUN'}, inplace=True)

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAJT_FINAL_RESULT.csv
c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAKT_MBO_USER.csv


In [18]:
point_data = pd.merge(wpe_data, mbo_data, on=['ID_SABUN', 'DT_YEAR'], how='outer')
point_data.dropna(subset = ['NO_POINT_TOTAL_FIN', 'DS_POINT_TOTAL_FIN', 'NO_FINAL_POINT', 'DS_FINAL_POINT'], axis=0, how='all', inplace=True)
point_data['ID_SABUN'] = point_data['ID_SABUN'].astype('str')

In [19]:
one_years = []

for i in range(len(bt_data)) : 
    temp_list = []
    temp_data = point_data[point_data['ID_SABUN']==bt_data['ID_SABUN'][i]]
    main = bt_data['DT_YEAR'][i] - 1
    if main not in temp_data['DT_YEAR'].unique().tolist() : 
        temp_list.append(0)
        temp_list.append(0)
    else : 
        if ((temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0]==temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0]) |
        (temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0]==temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0])) : 
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['NO_POINT_TOTAL_FIN'].values[0])
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['DS_POINT_TOTAL_FIN'].values[0])
        else : 
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['NO_FINAL_POINT'].values[0])
            temp_list.append(temp_data[temp_data['DT_YEAR']==main]['DS_FINAL_POINT'].values[0])  
            
    one_years.append(temp_list) 
    
bt_data[['FINAL_POINT_RECENT_YEAR', 'FINAL_GRADE_RECENT_YEAR']] = one_years

### DACT_WORK_RESULT_UPDATE

In [7]:
print(file_list[6])
work = pd.read_csv(file_list[6])
work_data = work.copy()

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DACT_WORK_RESULT_UPDATE.csv


In [21]:
year = []
month = [] 
for i in range(len(work_data)) : 
    year.append(int(str(work_data['DT_VACAFROM'][i])[0:4]))
    month.append(int(str(work_data['DT_VACAFROM'][i])[4:6]))
    
work_data['DT_YEAR'] = year
work_data['DT_MONTH'] = month
work_data['ID_SABUN'] = work_data['ID_SABUN'].astype('str')

In [22]:
work_result = [] 

for i in range(len(bt_data)) : 
    work_month = [0, 0, 0, 0, 0, 0]
    if bt_data['TEMP'][i] == 'train' : 
        temp_data = work_data[work_data['ID_SABUN']==bt_data['ID_SABUN'][i]]
        point_list = temp_data['DT_MONTH'].value_counts()
        for point in point_list.index : 
            if point <= 6 : 
                work_month[point-1] = point_list[point]
    else : 
        temp_data = work_data[work_data['ID_SABUN']==bt_data['ID_SABUN'][i]]
        point_list = temp_data['DT_MONTH'].value_counts()
        for point in point_list.index : 
            if point >= 7 : 
                work_month[point-7] = point_list[point]
            
    work_result.append(work_month)
    
bt_data[['WEAK_ONE', 'WEAK_TWO', 'WEAK_THREE', 'WEAK_FOUR', 'WEAK_FIVE', 'WEAK_SIX']] = work_result

### DAET_EDUCATION_REQUEST

In [8]:
print(file_list[7])
edu = pd.read_csv(file_list[7])
edu_data = edu.copy()

c:\Users\wongi\Desktop\알고리즘랩스\프로젝트\현대산업개발\develop_server\algorithmlabs.csv\DAET_EDUCATION_REQUEST.csv


In [24]:
edu_list = []

for id in edu_data['ID_SABUN'].unique().tolist() : 
    first = 0
    second = 0
    temp_data = edu_data[edu_data['ID_SABUN']==id]
    for i in range(len(temp_data)) : 
        if int(str(temp_data['DT_START'].values[i])[4:6]) in [1, 2, 3, 4, 5, 6] : 
            first += 1 
        else : 
            second += 1
    edu_list.append([id, first, second])

In [25]:
edu_temp = pd.DataFrame(edu_list, columns=['ID_SABUN', 'FIRST', 'SECOND'])
edu_temp['ID_SABUN'] = edu_temp['ID_SABUN'].astype('str')

edu_cnt = []
for i in range(len(bt_data)) : 
    if bt_data['ID_SABUN'][i] not in edu_temp['ID_SABUN'].unique().tolist() : 
        edu_cnt.append(0)
    else : 
        if bt_data['TEMP'][i] == 'train' : 
            edu_cnt.append(edu_temp[edu_temp['ID_SABUN']==bt_data['ID_SABUN'][i]]['FIRST'].values[0])
        else : 
            edu_cnt.append(edu_temp[edu_temp['ID_SABUN']==bt_data['ID_SABUN'][i]]['SECOND'].values[0])
        
bt_data['EDU_CNT'] = edu_cnt

### missing values

In [26]:
missing_data = bt_data[bt_data['TEMP']=='train']

label_list = []

for i in range(len(missing_data)) : 
    if (missing_data['DS_RETIRE'][i] == '재직') : 
        label_list.append(0)
    else : 
        if missing_data['DT_RETIRE'][i] > 20220630 and missing_data['DT_RETIRE'][i] < 20230101 : 
            label_list.append(1)
        elif missing_data['DT_RETIRE'][i] >= 20230101 :
            label_list.append(0)
        else : 
            label_list.append(-1)
    
    
missing_data['LABEL'] = label_list

missing_data = missing_data[missing_data['LABEL']!=-1]
missing_data = missing_data[missing_data['DT_JOIN'] <= 20220630]

missing_data.drop(['ID_SABUN', 'DT_RETIRE', 'DS_RETIRE', 'FINAL_POINT_RECENT_YEAR', 'TEMP', 'DT_YEAR', 
                   'DS_JOBTYPE', 'DS_POSITION', 'DS_DEPT', 'DS_BONBU', 'DS_OCCUPATION', 'DS_DUTY', 'DS_ADOPTYPE', 'DS_JOBFAMILY', 'DS_JOBFAMILY_JOIN', 'DS_TY_DEPT',
                   'DS_HNAME', 'DS_HANDPHONE', 'DS_EMAIL'], axis=1, inplace=True)
missing_rate = (missing_data.isnull().sum()/len(missing_data)).to_dict()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20296\1326882047.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_data['LABEL'] = label_list


In [27]:
bt_data['CD_JOBTYPE'] = bt_data['CD_JOBTYPE'].fillna(bt_data['CD_JOBTYPE'].mode()[0])
bt_data['CD_JOBFAMILY'] = bt_data['CD_JOBFAMILY'].fillna(bt_data['CD_JOBFAMILY'].mode()[0])
bt_data['CD_DUTY'] = bt_data['CD_DUTY'].fillna(bt_data['CD_DUTY'].mode()[0])

In [28]:
bt_data['YN_HANDICAP'] = bt_data['YN_HANDICAP'].fillna('N')
bt_data['YN_SUPPORTED'] = bt_data['YN_SUPPORTED'].fillna('N')
bt_data['YN_FOREIGNER'] = bt_data['YN_FOREIGNER'].fillna('N')
bt_data['YN_GENDER'] = bt_data['YN_GENDER'].fillna('M')

In [29]:
def ch_grade(data, columns) : 
    data[columns] = data[columns].replace({'O' : 3, 'R' : 2, 'D' : 1, 0 : 2, 'A' : 3, 'B' : 2, 'C' : 1})
    data[columns] = data[columns].fillna(2)
    return data

In [30]:
ch_grade(bt_data, 'FINAL_GRADE_RECENT_YEAR')

,TEMP,ID_SABUN,DT_YEAR,DT_GROUPJOIN,DT_JOIN,CD_JOBTYPE,CD_POSITION,CD_DEPT,CD_BONBU,TY_DEPT,...,VAT_ONE_MONTH_AGO,FINAL_POINT_RECENT_YEAR,FINAL_GRADE_RECENT_YEAR,WEAK_ONE,WEAK_TWO,WEAK_THREE,WEAK_FOUR,WEAK_FIVE,WEAK_SIX,EDU_CNT
0,train,2731,2022,19880125.0,19880125,45.0,12.0,50282364,605,3.0,...,0,70.881,1,0,0,0,0,0,0,0
1,train,2801,2022,19880704.0,19880704,32.0,12.0,50282389,605,3.0,...,0,72.712,1,3,0,0,0,0,0,1
2,train,2837,2022,19881010.0,19881010,45.0,12.0,50297026,605,3.0,...,0,71.902,1,0,0,0,0,0,0,0
3,train,2839,2022,19881010.0,19881010,45.0,12.0,50257954,211,3.0,...,0,77.65,2,0,0,0,0,0,0,0
4,train,2841,2022,19881010.0,19881010,45.0,12.0,50282175,605,3.0,...,0,78.245,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,test,5207,2022,19990608.0,19990608,14.0,11.0,50301764,211,1.0,...,0,NaN,3,0,0,0,0,0,0,0
2189,test,5727,2022,20051219.0,20051219,49.0,12.0,50279104,505,1.0,...,2,96.821,3,0,0,0,0,0,0,0
2190,test,6158,2022,20110701.0,20110701,24.0,13.0,50298128,505,1.0,...,1,85.817,2,0,1,0,0,0,0,0
2191,test,6446,2022,19960122.0,20191101,44.0,11.0,50063156,605,1.0,...,0,NaN,2,0,0,0,0,0,0,0


In [31]:
birth_change = {' ' : '서울', '대한민국' : '서울', '충청남도' : '충남', '광주광역시' : '광주', ' 서울' : '서울', '강원도' : '강원', '상주' : '경북', '구미' : '경북',
                '충남 서산' : '충남', '경기도 성남시' : '경기', '수원' : '경기', '한국' : '서울', '파라과이' : '해외', '대한민국' : '서울', '전북 익산' : '전북', 
                '강릉' : '강원', '경북 대구' : '대구', '포항' : '경북', '청주' : '충북', '경기도 안양시' : '경기', '부산광역시' : '부산', '전북 전주' : '전북',
                '전남 광주' : '광주', '경남 통영시' : '경남', '경상남도' : '경남', '경기도 수원시' : '경기', '경기 광명' : '경기', '강원도 홍천' : '강원',
                '경북 영덕군' : '경북', ' 충주 ' : '충북', '경기도 수원' : '경기', '인천광역시' : '인천', '서울특별시' : '서울', '전북 남원' : '전북'}

bt_data['DS_BIRTHPLACE'].replace(birth_change, inplace=True)
bt_data['DS_BIRTHPLACE'] = bt_data['DS_BIRTHPLACE'].fillna(bt_data['DS_BIRTHPLACE'].mode()[0])

In [32]:
train_data = bt_data[bt_data['TEMP']=='train']
inference_data = bt_data[bt_data['TEMP']=='test']

In [33]:
# feature : 20220630 / label : 20230101 / v4
# feature : 20220630 / label : 20221001 / v5
# feature : 20220331 / label : 20220630 / v6
# feature : 20220930 / label : 20230101 / v7

label_list = []

for i in range(len(train_data)) : 
    if (train_data['DS_RETIRE'][i] == '재직') : 
        label_list.append(0)
    else : 
        if train_data['DT_RETIRE'][i] > 20220630 and train_data['DT_RETIRE'][i] < 20230101 : 
            label_list.append(1)
        elif train_data['DT_RETIRE'][i] >= 20230101 :
            label_list.append(0)
        else : 
            label_list.append(-1)
    
    
train_data['LABEL'] = label_list

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20296\2033796102.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['LABEL'] = label_list


In [34]:
train_data = train_data[train_data['LABEL']!=-1]
train_data = train_data[train_data['DT_JOIN'] <= 20220630]

In [35]:
train_data.drop(['DT_RETIRE', 'DS_RETIRE', 'FINAL_POINT_RECENT_YEAR', 'TEMP', 'DT_YEAR', 'DS_HNAME', 'DS_HANDPHONE', 'DS_EMAIL'], axis=1, inplace=True)
inference_data.drop(['DT_RETIRE', 'DS_RETIRE', 'FINAL_POINT_RECENT_YEAR', 'TEMP', 'DT_YEAR'], axis=1, inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20296\2148771533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inference_data.drop(['DT_RETIRE', 'DS_RETIRE', 'FINAL_POINT_RECENT_YEAR', 'TEMP', 'DT_YEAR'], axis=1, inplace=True)


In [36]:
name_dict = {}
for i in range(len(name)) : 
    name_dict[name['feature'][i]] = name['feature_name'][i]

In [37]:
today = datetime.today().strftime("%Y%m%d")
Path(f"/ml/algorithmlabs.inference/{today}/attrition/pickle").mkdir(parents=True, exist_ok=True)

train_data.to_csv(f"/ml/algorithmlabs.inference/{today}/attrition/train_data_attrition.csv", index=False)
inference_data.to_csv(f"/ml/algorithmlabs.inference/{today}/attrition/inference_data_attrition.csv", index=False)
joblib.dump(missing_rate, f"/ml/algorithmlabs.inference/{today}/attrition/pickle/missing_rate.pkl")
joblib.dump(name_dict, f"/ml/algorithmlabs.inference/{today}/attrition/pickle/name_dict.pkl")

['c:\\Users\\wongi\\Desktop\\알고리즘랩스\\프로젝트\\현대산업개발\\develop\\algorithmlabs.inference\\pickle_attrition\\name_dict.pkl']